In [1]:
import pandas as pd
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.keras as krs
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tqdm.autonotebook import tqdm
from tqdm import trange

/Users/filipwojcik/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  del sys.path[0]


In [2]:
%matplotlib inline

In [3]:
%load_ext autoreload

%autoreload 2

In [4]:
if os.getcwd().endswith("notebooks"):
    os.chdir(os.pardir)

In [5]:
sys.path.append(os.getcwd())

In [6]:
from src.data.train_data_utils import prepare_experiment_data
from src.models.training_utils import generator_ratings, generator_ratings_features, mae_masked, mape_masked, mse_masked
from src.models.train_model import get_collabfiltering_model1, build_shallow_autorec_single_input, build_autorec_multi_input2, get_array

# Data prep

In [9]:
exd = prepare_experiment_data()

Processed: 0.0%
Processed: 0.27373261797875836%
Processed: 0.5474652359575167%
Processed: 0.8211978539362751%
Processed: 0.0%
Processed: 0.4555393586005831%
Processed: 0.9110787172011662%
Processed: 0.0%
Processed: 0.6804109682248077%


# Model prep

In [76]:
dim_embed = 30
nusers = max(exd.Xraw_train.user_id.max(), exd.Xraw_test.user_id.max(), exd.Xraw_valid.user_id.max())
nitems = exd.Xraw_train.item_id.max()

f_input = krs.Input(shape=(exd.Xfeatures_train.shape[1],), name="features_input", dtype='float32')
user_input = krs.Input(shape=(1,), name="user_input", dtype='int32')
u = krs.layers.Embedding(nusers + 1, dim_embed)(user_input)

item_input = krs.Input(shape=(1,), name='item_input', dtype='int32')
i = krs.layers.Embedding(nitems + 1, dim_embed)(item_input)

ui = krs.layers.Dot(axes=(2,2))([u, i])
ui = krs.layers.Flatten()(ui)
all_features = krs.layers.Concatenate()([f_input, ui])
drop = krs.layers.Dropout(0.4)(all_features)

d1 = krs.layers.Dense(32, activation="relu")(all_features)
drop2 = krs.layers.Dropout(0.4)(d1)
d2 = krs.layers.Dense(16, activation="relu")(drop2)
out = krs.layers.Dense(1)(d2)

model = krs.Model(inputs=[user_input, item_input, f_input], outputs=out)
model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae', 'mape'])

In [77]:
print(model.summary())

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_42 (Embedding)        (None, 1, 30)        610050      user_input[0][0]                 
__________________________________________________________________________________________________
embedding_43 (Embedding)        (None, 1, 30)        196680      item_input[0][0]                 
___________________________________________________________________________________________

In [78]:
x = [
        exd.Xraw_train.user_id.to_numpy(),
        exd.Xraw_train.item_id.to_numpy(),
        exd.Xfeatures_train
    ]
y = exd.Xraw_train.rating.to_numpy()

x_val = [
    exd.Xraw_valid.user_id.to_numpy(),
    exd.Xraw_valid.item_id.to_numpy(),
    exd.Xfeatures_valid
]
y_val = exd.Xraw_valid.rating.to_numpy()


In [79]:
model_history = model.fit(x=x, y=y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Train on 36532 samples, validate on 14697 samples
Epoch 1/10
36532/36532 [==============================] - 16s 447us/sample - loss: 2.8657 - mse: 2.8657 - mae: 1.2843 - mape: 45.5651 - val_loss: 1.3984 - val_mse: 1.3984 - val_mae: 0.9624 - val_mape: 38.0863
Epoch 2/10
36532/36532 [==============================] - 15s 413us/sample - loss: 1.5606 - mse: 1.5606 - mae: 0.9765 - mape: 39.5114 - val_loss: 1.4021 - val_mse: 1.4021 - val_mae: 0.9580 - val_mape: 38.3236
Epoch 3/10
36532/36532 [==============================] - 14s 387us/sample - loss: 0.7747 - mse: 0.7747 - mae: 0.6781 - mape: 26.5813 - val_loss: 1.4773 - val_mse: 1.4773 - val_mae: 1.0175 - val_mape: 38.5115
Epoch 4/10
36532/36532 [==============================] - 14s 397us/sample - loss: 0.3786 - mse: 0.3786 - mae: 0.4781 - mape: 17.0270 - val_loss: 1.3974 - val_mse: 1.3974 - val_mae: 0.9498 - val_mape: 38.4423
Epoch 5/10
36532/36532 [==============================] - 14s 376us/sample - loss: 0.1844 - mse: 0.1844 - mae: 0.3

In [87]:
yhat = model.predict(x=[
    exd.Xraw_test.user_id.to_numpy(dtype='int32'),
    exd.Xraw_test.item_id.to_numpy(dtype='int32'),
    exd.Xfeatures_test.astype('float32')
])

In [33]:
import gc
tf.keras.backend.clear_session()
gc.collect()

1631